In [1]:
import random
import string
import time
from IPython.display import clear_output

In [2]:
"""
#############################################################
Functions to be used in blackjack

#############################################################
"""                   
def player_creator(player = None):
    
    name = input("Player name: ")
    balance = input("Player balance: ")
    
    while True:
        test_string = ""
        for character in balance:
            if character in string.digits:
                test_string+= character
        if test_string == balance:
            new_user = Player(name,abs(int(balance)))
            break
        else:
            balance = input("Please enter a valid number balance: ")
        
  
    return new_user

def card_counter(card_list):
    face_cards = ["J","K","Q",]
    count = 0 
    for i in card_list:
        if i == "A" and count <=10:
            count+=11
        elif i == "A" and count >10:
            count +=1
        elif i in face_cards:
            count+= 10
        else:
            count+=i
    if "A" in card_list and count > 21:
        count -=10
    return count
    



def rules():
    print("""
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    Rules for this version of BlackJack
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
      # The dealer will hit until the total of his cards are > 16
      
      # If you would like to stop playing, type "quit" when asked for a bet value, or wait until the end of a hand.
    
    
    
    """)

def table_printer(hand1,hand2):
     print("""
        *********************************
        **  dealer cards: {c1}         
        **                             
        **                             
        **                             
        **                             
        **  Your cards: {c2}           
        *********************************
        """.format(c1=hand1,c2 = hand2))

def replay():
    play_again = input("Would you like to play again? (Y/N)")
    
    while play_again.lower().startswith("y") == False and play_again.lower().startswith("n") == False:
        play_again = input("Please type 'Yes' to play again, or 'No' to quit.")
    if play_again.lower().startswith("y"):
        return True
    else:
        return False
    
    



In [3]:
"""
########################################################################
Classes to be used in blackjack

###########################################################################
"""




class Player():
    '''
    This class creates a player that can play BlackJack. The expected input is a name and a balance to play with.
    '''
    def __init__(self,name,balance):
        self.name = name
        self.balance = balance
        
        
        
    def bet(self,amount):
        self.bet_amount = amount
        
        #Betting function
        while type(amount) != int:
            amount = int(input("Please make sure you are betting a number of dollars. Try again "))
            
        if self.balance==0:
            return "You must make a deposit before you can bet"
        
        while amount > self.balance or amount <= 0:
            print("You must enter a valid amount that does not exceed your balance.")
            amount = int(input("New bet value: "))
        self.balance -= amount
            
    
            
    def deposit(self,amount):
        self.balance += amount
        print("Your balance is now {} dollars".format(self.balance))
        
    
        
        
        
        
        
        

class Deck():
    """
    This class creates a deck of cards and has actions you can do with the cards
    """
    def __init__(self):
        self.card_types = ["A",2,3,4,5,6,7,8,9,10,"J","Q","K"]
        self.full_deck = 4*self.card_types
    
    
    #Deal
    def deal(self):
        self.index_number = random.randint(0,(len(self.full_deck))-1)
        self.chosen_card = self.full_deck[self.index_number]
        self.full_deck.pop(self.index_number)
        return self.chosen_card
    
    #shuffle
    def shuffle(self):
        self.full_deck = 4*self.card_types
        return self.full_deck
        
        
        
        
    

"""
Class defines a player's hand. Can be dealer
"""


class PlayerHand(Deck):
    def __init__(self):
        self.card_types = ["A",2,3,4,5,6,7,8,9,10,"J","Q","K"]
        self.full_deck = 4*self.card_types
        self.player_hand = []
        
        while len(self.player_hand) <2:
            self.player_hand.append(super().deal())
            
        self.dealer_hand = ["X"]+self.player_hand[1:]
        
    def hit(self,card=None):
        card = super().deal()
        self.player_hand.append(card)
        self.dealer_hand.append(card)
        
    def dealer_play(self):
        self.dealer_count = card_counter(self.player_hand)
        self.loop_count = 0
        while self.dealer_count <17:
            self.hit()
            self.dealer_count = card_counter(self.player_hand)
            self.loop_count+=1
        print("The dealer hit {} time(s)".format(self.loop_count))
        return self.dealer_count
            
        
    
        

In [4]:
"""
THIS FUNCTION WILL PLAY BLACKJACK
"""


def black_jack(player = None):
    rules()
    
    if player == None:
        player = player_creator()
    print("""
        @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        HELLO {}, WELCOME TO BLACKJACK. 
        @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        """.format(player.name.capitalize()))
    
    #Game start
    
    game = True
    loss = None
    while game == True:
        
        if player.balance == 0:
            print("Your balance is 0. If you would like to play, please make a deposit.")
            player.deposit(int(input("Amount to deposit: ")))
        
        
        decision1 = input("How much would you like to bet? ")
        if decision1.lower() == "quit":
            clear_output()
            print("Thank you for playing, your final balance is {} dollars.".format(player.balance))
            
        else:

            player.bet(int(decision1))
            player_hand = PlayerHand()
            dealer_hand = PlayerHand()
            player_hand.full_deck = player_hand.shuffle()
            dealer_hand.full_deck = dealer_hand.shuffle()

        #Player plays

            table_printer(dealer_hand.dealer_hand,player_hand.player_hand)
            player_total = card_counter(player_hand.player_hand)
            if player_total !=21:
                hit_or_stay = input("Would you like to hit or stay? ")
                while hit_or_stay.lower() == "hit":
                    player_hand.hit()
                    table_printer(dealer_hand.dealer_hand,player_hand.player_hand)
                    player_total = card_counter(player_hand.player_hand)
                    if player_total >21:
                        print("Bust! You lose, better luck next time.")
                        print("Your balance is now {}".format(player.balance))
                        game = replay()
                        loss = True
                        break
                    elif player_total == 21:
                        break
                    else: 
                        hit_or_stay = input("Would you like to hit or stay? ")

            
            #Dealer Plays
            if loss != True:
                dealer_hand.dealer_play()
                table_printer(dealer_hand.player_hand,player_hand.player_hand)
                dealer_total = card_counter(dealer_hand.player_hand)
                if dealer_total > 21:
                    print("Dealer Bust. You win!")
                    player.balance+= 2*int(decision1)
                    print("Your balance is now {}".format(player.balance))
                    game = replay()
                elif dealer_total == player_total:
                    print("This round was a Tie!")
                    player.balance+= int(decision1)
                    print("Your balance is now {}".format(player.balance))
                    game = replay()
                elif player_total > dealer_total:
                    print("You win!")
                    player.balance+= 2*int(decision1)
                    print("Your balance is now {}".format(player.balance))
                    game = replay()
                else:
                    print("You lose, better luck next time.")
                    print("Your balance is now {}".format(player.balance))
                    game = replay()

                
    clear_output()
    print("Your final balance is {} dollars".format(player.balance))
    return player

    
            
        
        
        


Your final balance is 0 dollars


NameError: name 'john' is not defined